In [4]:
# Import useful libraries
import numpy as np
from ci_test import ci_test
from scipy.io import loadmat
import networkx as nx
from itertools import chain, combinations, permutations
from helpers import *
from collections import deque
from copy import copy
from matplotlib import pyplot as plt
from networkx.algorithms.connectivity import minimum_st_node_cut


from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
nodes = ['z','t','s','w']
G_b = nx.Graph()
G_d = nx.DiGraph()
for node in nodes:
    G_b.add_node(node)
    G_d.add_node(node)
G_d.add_edge('z','t')
G_d.add_edge('w','t')
G_d.add_edge('t','s')

G_b.add_edge('z','s')
G_b.add_edge('z','t')
G_b.add_edge('t','w')
G_b.add_edge('w','s')
HHull(G_d, G_b, {'s'})

{'s', 't', 'w', 'z'}

In [9]:
costs = {'z':2, 'w':2, 't':3, 's':1}

MinCostIntervention({'s'},G_d, G_b,costs)

{'t'}

In [16]:
def min_nodes_cut(G,source_set,S,weights):
    
    weighted_edges_graph = nx.DiGraph()

    for node in source_set:
        weighted_edges_graph.add_edge('x',str(node)+'_in',capacity = np.inf)
    
    for node in S:
        weighted_edges_graph.add_edge(str(node)+'_in',str(node)+'_out',capacity = np.inf)
        weighted_edges_graph.add_edge(str(node)+'_out','y',capacity = np.inf)

    for node in G.nodes - S:
        weighted_edges_graph.add_edge(str(node)+'_in',str(node)+'_out',capacity=weights[node])
        for neighbor in G.adj[node]:
            weighted_edges_graph.add_edge(str(node)+'_out',str(neighbor)+'_in',capacity = np.inf)

    cost, cut_sets = nx.minimum_cut(weighted_edges_graph,'x','y')

    setA , setB = cut_sets

    cut_set = set()
    for node in G.nodes - S:
        if (str(node)+'_in' in setA and str(node)+'_out' in setB) or (str(node)+'_in' in setB and str(node)+'_out' in setA):
            cut_set = cut_set.union(node)
            
    return (cost,cut_set)

In [61]:
def heuristic_algorithm(G_directed, G_bidirected, S, costs): # taken from Sina's paper
    
    # we need to move from min weith vertex cut to min weight edge cut and solve it with min flow max cut
    
    H = HHull(G_directed, G_bidirected, S)
    
    #cal_H, pa_inter_H, pa_double = initialize_cal_H(G_directed, G_bidirected, S)
    
    new_costs = {key:value for key,value in costs.items() if key not in S}
    
    pa_S = set()
    
    for node in S:
        pa_S = pa_S.union(set(G_directed.predecessors(node)))
    
    pa_inter_H = H & pa_S
    
    cost, cut_set = min_nodes_cut(G_bidirected.subgraph(H), pa_inter_H, S, new_costs)
    
    return A
    
    
    
    
    
    

In [62]:
gd = nx.DiGraph()
gu = nx.Graph()


# Add nodes
gd.add_nodes_from([ 'z', 't', 's','w'])
gu.add_nodes_from([ 'z', 't', 's','w'])


# Add edges
gd.add_edges_from([('z', 't'), ('w','t'),('t', 's')])
gu.add_edges_from([('z', 't'),('w','t',),('w','s'),('z','s')])

weights = {'z':2,'w':2,'t':3}
#MinCostIntervention(S, gd, gu, weights)

S = {'s'}
H = HHull(gd,gu,S)
print(H)
H1 = H & set(gd.predecessors('s'))
print(H1)

cost,A = min_nodes_cut(gu,H1,S, weights)

#Sanity check
gd_test = gd.copy()
for node in A:
    parents = list(gd_test.predecessors(node))

    # Remove edges from the parents
    for parent in parents:
        gd_test.remove_edge(parent, node)
        
#HHull(S,gd_test,gu)
print(A)

{'z', 't', 's', 'w'}
{'t'}
{'t'}


In [63]:
heuristic_algorithm(G_d, G_b, {'s'}, costs)

{'t'}